In [97]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data


In [98]:
import requests

In [99]:
# itertools has lots of things that deal with looping
# "generators" and "iterators" are really nice concepts that
#can be used in place of most usage of using indexes in a loop
#because usually we aren't actually interested in counting the lines
from itertools import islice
data = []
with open('amateur.txt') as f:
    #we don't know we've reached the end of the file
    #until we actually reached it
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        # this is called a "list comprehension"
        # it is pretty much exactly the same as
        #    tle = []
        #    for line in tripleline:
        #      tle.append(x.rstrip())
        #
        tle = [x.rstrip() for x in tripleline]
        
        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break

# now, data is a list of 3-element lists of strings

In [100]:
# get NORAD ID for each sat
print(data[0][2][2:7])
print(data[1][2][2:7])

07530
14781


In [101]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 12:00:00'
num_passes = 10
duration = None
vu_passes = {}
i = 0

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    vu_passes[data[i][2][2:7]] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)
    i = i + 1

print(vu_passes['25544'])

[{'rise_az': 190.456669458409, 'end': 'Thu Jun  8 19:35:05 2017', 'duration': '0:09:29.955764', 'set_az': 70.48850846101539, 'start': 'Thu Jun  8 19:25:35 2017'}, {'rise_az': 238.95741928294103, 'end': 'Thu Jun  8 21:11:59 2017', 'duration': '0:11:03.015994', 'set_az': 53.80764288509183, 'start': 'Thu Jun  8 21:00:56 2017'}, {'rise_az': 277.83698705767466, 'end': 'Thu Jun  8 22:48:30 2017', 'duration': '0:10:11.354298', 'set_az': 52.31765419312647, 'start': 'Thu Jun  8 22:38:18 2017'}, {'rise_az': 302.7234369967764, 'end': 'Fri Jun  9 00:25:45 2017', 'duration': '0:09:49.092054', 'set_az': 68.37534355446182, 'start': 'Fri Jun  9 00:15:56 2017'}, {'rise_az': 308.7222555410152, 'end': 'Fri Jun  9 02:03:28 2017', 'duration': '0:10:44.866753', 'set_az': 101.9204174741996, 'start': 'Fri Jun  9 01:52:43 2017'}, {'rise_az': 299.42355404268983, 'end': 'Fri Jun  9 03:39:58 2017', 'duration': '0:10:43.449490', 'set_az': 145.65946676791137, 'start': 'Fri Jun  9 03:29:14 2017'}, {'rise_az': 266.29

In [102]:
import pprint
pprint.pprint(vu_passes)

{'07530': [{'duration': '0:12:41.902490',
            'end': 'Thu Jun  8 13:18:54 2017',
            'rise_az': 3.1217117066147058,
            'set_az': 291.59949038617015,
            'start': 'Thu Jun  8 13:06:12 2017'},
           {'duration': '0:09:47.030825',
            'end': 'Thu Jun  8 16:52:59 2017',
            'rise_az': 53.02774114980851,
            'set_az': 358.85267484113393,
            'start': 'Thu Jun  8 16:43:12 2017'},
           {'duration': '0:19:05.962921',
            'end': 'Thu Jun  8 18:47:04 2017',
            'rise_az': 110.45268295519035,
            'set_az': 351.6644744755858,
            'start': 'Thu Jun  8 18:27:58 2017'},
           {'duration': '0:22:33.247218',
            'end': 'Thu Jun  8 20:40:30 2017',
            'rise_az': 158.72548204636234,
            'set_az': 343.69645775211893,
            'start': 'Thu Jun  8 20:17:56 2017'},
           {'duration': '0:19:14.097460',
            'end': 'Thu Jun  8 22:32:34 2017',
            'rise

'Thu Jun  8 20:53:55 2017',
            'rise_az': 183.11539089318254,
            'set_az': 37.931551178314194,
            'start': 'Thu Jun  8 20:38:42 2017'},
           {'duration': '0:15:29.809790',
            'end': 'Thu Jun  8 22:37:25 2017',
            'rise_az': 232.95392888930994,
            'set_az': 24.102494486546057,
            'start': 'Thu Jun  8 22:21:55 2017'},
           {'duration': '0:11:21.285834',
            'end': 'Fri Jun  9 00:20:26 2017',
            'rise_az': 284.96581941895704,
            'set_az': 17.062531424101994,
            'start': 'Fri Jun  9 00:09:05 2017'},
           {'duration': '0:06:49.112288',
            'end': 'Fri Jun  9 02:04:55 2017',
            'rise_az': 332.8518931993229,
            'set_az': 24.6941169347161,
            'start': 'Fri Jun  9 01:58:06 2017'},
           {'duration': '0:10:52.175822',
            'end': 'Fri Jun  9 03:53:47 2017',
            'rise_az': 343.1746859410433,
            'set_az': 70.486104234427

,
           {'duration': '0:10:47.032665',
            'end': 'Fri Jun  9 10:35:27 2017',
            'rise_az': 304.32809431962255,
            'set_az': 201.04839406850954,
            'start': 'Fri Jun  9 10:24:40 2017'},
           {'duration': '0:12:55.168491',
            'end': 'Fri Jun  9 23:10:46 2017',
            'rise_az': 180.87532107185015,
            'set_az': 45.09777541020782,
            'start': 'Fri Jun  9 22:57:51 2017'},
           {'duration': '0:13:40.164527',
            'end': 'Sat Jun 10 00:51:25 2017',
            'rise_az': 232.0907842235171,
            'set_az': 27.881872088293186,
            'start': 'Sat Jun 10 00:37:45 2017'}],
 '27844': [{'duration': '0:15:44.640535',
            'end': 'Thu Jun  8 12:33:52 2017',
            'rise_az': 12.871590528631994,
            'set_az': 195.95166592871098,
            'start': 'Thu Jun  8 12:18:07 2017'},
           {'duration': '0:12:22.125564',
            'end': 'Thu Jun  8 14:11:27 2017',
            'r

: 'Fri Jun  9 16:11:22 2017'},
           {'duration': '0:08:55.017829',
            'end': 'Sat Jun 10 02:25:40 2017',
            'rise_az': 42.76260203487216,
            'set_az': 133.54537495408505,
            'start': 'Sat Jun 10 02:16:45 2017'}],
 '33499': [{'duration': '0:11:26.776351',
            'end': 'Thu Jun  8 21:36:09 2017',
            'rise_az': 33.215162122447815,
            'set_az': 149.122058488406,
            'start': 'Thu Jun  8 21:24:42 2017'},
           {'duration': '0:13:28.485152',
            'end': 'Thu Jun  8 23:14:04 2017',
            'rise_az': 8.370123620283826,
            'set_az': 205.5561413357995,
            'start': 'Thu Jun  8 23:00:35 2017'},
           {'duration': '0:08:27.131675',
            'end': 'Fri Jun  9 00:47:07 2017',
            'rise_az': 343.4662803770895,
            'set_az': 265.09928531224784,
            'start': 'Fri Jun  9 00:38:39 2017'},
           {'duration': '0:11:11.097861',
            'end': 'Fri Jun  9 08:34

'Fri Jun  9 21:11:21 2017',
            'rise_az': 12.290757218029635,
            'set_az': 196.55303676437393,
            'start': 'Fri Jun  9 20:56:57 2017'},
           {'duration': '0:10:51.892045',
            'end': 'Fri Jun  9 22:46:36 2017',
            'rise_az': 351.4297891757042,
            'set_az': 251.84382790697737,
            'start': 'Fri Jun  9 22:35:44 2017'}],
 '35933': [{'duration': '0:13:01.480360',
            'end': 'Thu Jun  8 20:08:08 2017',
            'rise_az': 27.906086816448173,
            'set_az': 158.87537737788745,
            'start': 'Thu Jun  8 19:55:06 2017'},
           {'duration': '0:14:00.169037',
            'end': 'Thu Jun  8 21:46:34 2017',
            'rise_az': 6.197741827399897,
            'set_az': 212.8540755205531,
            'start': 'Thu Jun  8 21:32:34 2017'},
           {'duration': '0:07:55.207622',
            'end': 'Thu Jun  8 23:20:04 2017',
            'rise_az': 341.78884055844776,
            'set_az': 273.404112324

...

: 348.14075256201954,
            'start': 'Fri Jun  9 05:20:29 2017'},
           {'duration': '0:09:20.156196',
            'end': 'Fri Jun  9 07:08:12 2017',
            'rise_az': 226.44918457286994,
            'set_az': 318.8135048698722,
            'start': 'Fri Jun  9 06:58:51 2017'},
           {'duration': '0:11:23.951516',
            'end': 'Fri Jun  9 17:17:43 2017',
            'rise_az': 31.4429807498288,
            'set_az': 151.5088817542453,
            'start': 'Fri Jun  9 17:06:19 2017'},
           {'duration': '0:13:00.992926',
            'end': 'Fri Jun  9 18:54:41 2017',
            'rise_az': 6.364604202596954,
            'set_az': 208.11724370850087,
            'start': 'Fri Jun  9 18:41:40 2017'},
           {'duration': '0:07:18.985017',
            'end': 'Fri Jun  9 20:26:40 2017',
            'rise_az': 338.83360894988,
            'set_az': 270.3514822179342,
            'start': 'Fri Jun  9 20:19:22 2017'},
           {'duration': '0:10:35.131898',

: 169.17819864647478,
            'set_az': 347.2201523449285,
            'start': 'Thu Jun  8 23:13:24 2017'},
           {'duration': '0:07:34.856126',
            'end': 'Fri Jun  9 00:58:05 2017',
            'rise_az': 234.76223513244915,
            'set_az': 313.0951612137108,
            'start': 'Fri Jun  9 00:50:30 2017'},
           {'duration': '0:09:46.692720',
            'end': 'Fri Jun  9 10:56:18 2017',
            'rise_az': 35.52206192971421,
            'set_az': 144.3396419938331,
            'start': 'Fri Jun  9 10:46:31 2017'},
           {'duration': '0:11:59.615885',
            'end': 'Fri Jun  9 12:31:54 2017',
            'rise_az': 6.739120550261679,
            'set_az': 204.55704858476673,
            'start': 'Fri Jun  9 12:19:54 2017'},
           {'duration': '0:06:32.802487',
            'end': 'Fri Jun  9 14:02:26 2017',
            'rise_az': 335.8761097185078,
            'set_az': 269.4682294752248,
            'start': 'Fri Jun  9 13:55:53 2017'

,
           {'duration': '0:05:31.845246',
            'end': 'Fri Jun  9 10:13:42 2017',
            'rise_az': 56.15738846902171,
            'set_az': 111.53700280668656,
            'start': 'Fri Jun  9 10:08:10 2017'},
           {'duration': '0:12:04.923473',
            'end': 'Fri Jun  9 11:51:59 2017',
            'rise_az': 16.02963777441411,
            'set_az': 183.5455015655966,
            'start': 'Fri Jun  9 11:39:54 2017'}],
 '40906': [{'duration': '0:10:50.450315',
            'end': 'Thu Jun  8 13:09:23 2017',
            'rise_az': 356.20671419818433,
            'set_az': 228.59539393462802,
            'start': 'Thu Jun  8 12:58:33 2017'},
           {'duration': '0:10:38.469803',
            'end': 'Thu Jun  8 22:16:29 2017',
            'rise_az': 127.71355453505306,
            'set_az': 5.449879655739459,
            'start': 'Thu Jun  8 22:05:51 2017'},
           {'duration': '0:11:54.236152',
            'end': 'Thu Jun  8 23:51:02 2017',
            'ris

: 221.14830207877972,
            'start': 'Fri Jun  9 17:29:21 2017'},
           {'duration': '0:03:43.623848',
            'end': 'Fri Jun  9 19:12:54 2017',
            'rise_az': 325.96654342157314,
            'set_az': 293.200158878541,
            'start': 'Fri Jun  9 19:09:10 2017'},
           {'duration': '0:01:19.133348',
            'end': 'Sat Jun 10 01:23:35 2017',
            'rise_az': 54.943827778050526,
            'set_az': 43.45870759432318,
            'start': 'Sat Jun 10 01:22:16 2017'}],
 '41338': [{'duration': '0:09:44.240294',
            'end': 'Fri Jun  9 02:43:13 2017',
            'rise_az': 204.2341035804302,
            'set_az': 109.7522129048597,
            'start': 'Fri Jun  9 02:33:29 2017'},
           {'duration': '0:11:52.523593',
            'end': 'Fri Jun  9 04:25:09 2017',
            'rise_az': 232.53482848183796,
            'set_az': 107.97568753192856,
            'start': 'Fri Jun  9 04:13:17 2017'},
           {'duration': '0:12:09.463

},
           {'duration': '0:06:07.011658',
            'end': 'Fri Jun  9 03:11:21 2017',
            'rise_az': 246.62870526810582,
            'set_az': 307.3282412521723,
            'start': 'Fri Jun  9 03:05:14 2017'},
           {'duration': '0:12:35.878101',
            'end': 'Fri Jun  9 13:16:29 2017',
            'rise_az': 28.391670577753764,
            'set_az': 157.33245130475993,
            'start': 'Fri Jun  9 13:03:54 2017'},
           {'duration': '0:13:41.668921',
            'end': 'Fri Jun  9 14:52:00 2017',
            'rise_az': 6.1160894771933325,
            'set_az': 211.03612505030583,
            'start': 'Fri Jun  9 14:38:18 2017'},
           {'duration': '0:08:03.530800',
            'end': 'Fri Jun  9 16:22:51 2017',
            'rise_az': 341.3649043770434,
            'set_az': 270.03393306304196,
            'start': 'Fri Jun  9 16:14:47 2017'},
           {'duration': '0:10:01.493698',
            'end': 'Sat Jun 10 00:01:20 2017',
            'r